In [ ]:
import cv2
import random

# Create a dictionary to store colors for each tid
color_dict = {}

def get_color_for_tid(tid):
    if tid not in color_dict:
        # Generate a random color
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        color_dict[tid] = color
    return color_dict[tid]

# Example usage within your loop
for detection in detections:  # Assuming detections is your list of detected objects
    tid = detection['tid']
    x, y, w, h = detection['bbox']  # Assuming bbox is in the format (x, y, width, height)
    
    # Calculate top-left and bottom-right points
    top_left = (int(x), int(y))
    bottom_right = (int(x + w), int(y + h))
    
    # Get the color for this tid
    color = get_color_for_tid(tid)
    
    # Draw the rectangle and the tid
    cv2.rectangle(frame, top_left, bottom_right, color, 2)
    cv2.putText(frame, str(tid), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Show the frame (this part is usually within your video processing loop)
cv2.imshow('Frame', frame)


In [4]:
import cv2
import numpy as np
import random

def scale_bbox(original_bbox, original_size, target_size):
    """
    Scales bounding box coordinates from original size to target size.
    
    :param original_bbox: Tuple (x, y, w, h) of the original bounding box
    :param original_size: Tuple (original_width, original_height)
    :param target_size: Tuple (target_width, target_height)
    :return: Scaled bounding box as a tuple (x, y, w, h)
    """
    x, y, w, h = original_bbox
    orig_w, orig_h = original_size
    targ_w, targ_h = target_size

    scale_x = targ_w / orig_w
    scale_y = targ_h / orig_h

    new_x = x * scale_x
    new_y = y * scale_y
    new_w = w * scale_x
    new_h = h * scale_y

    return new_x, new_y, new_w, new_h

def draw_bounding_boxes(video_path, txt_path, output_path):
    # Read the bounding box data from the text file
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    # Parse the bounding box data
    bbox_data = {}
    for line in lines:
        parts = line.strip().split(',')
        frame_id = int(parts[0])
        tid = int(parts[1])
        x, y, w, h = map(float, parts[2:6])
        if frame_id not in bbox_data:
            bbox_data[frame_id] = []
        bbox_data[frame_id].append((tid, x, y, w, h))

    # print(bbox_data)
    # return
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Get video properties
    target_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    target_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    original_width = 2048
    original_height = 1536

    print(original_width, original_height,target_width, target_height, fps)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
    out = cv2.VideoWriter(output_path, fourcc, 29.25, (target_width, target_height))
    frame_id = 0
    written = 0
    while cap.isOpened():
        ret, frame = cap.read()
        # if frame_id >= 3150 and frame_id <= 4050:
        if frame_id <= 4050:
            if not ret:
                break
            
            # Draw bounding boxes on the current frame
            if frame_id in bbox_data:
                for tid, x, y, w, h in bbox_data[frame_id]:
                    scaled_bbox = scale_bbox((x, y, w, h), (original_width, original_height), (target_width, target_height))
                    x_s, y_s, w_s, h_s = scaled_bbox
                    top_left = (int(x_s), int(y_s))
                    bottom_right = (int(x_s + w_s), int(y_s + h_s))
                    # Get the color for this tid
                    color = get_color_for_tid(tid)
                    cv2.rectangle(frame, top_left, bottom_right, color, 2)
                    cv2.putText(frame, str(tid), (int(x_s), int(y_s) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Write the frame to the output video
            out.write(frame)
            written += 1
        frame_id += 1

        if frame_id == 15699:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    print("Video processing completed")
    print(written)


video_path = 'path to video'
txt_path = 'path to modified bbox txt'
output_path = 'output path'
draw_bounding_boxes(video_path, txt_path, output_path)
print("done with video1")



2048 1536 2048 1536 30.0
Video processing completed
4051
done no video1
2048 1536 2048 1536 29.25
Video processing completed
4051
done no video2
2048 1536 2048 1536 30.0
Video processing completed
4051
done with video1
2048 1536 2048 1536 29.25
Video processing completed
4051
done with video2
